In [1]:
import glob
import json
from random import sample
import sklearn
import re
import string
import warnings
from bs4 import BeautifulSoup
import gensim
from gensim.models.phrases import Phrases
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from analysis_func.text_preproc import preproc_text
from analysis_func.topic_preproc import prepare_corp
from analysis_func.topic_model import topicmodel
from analysis_func.saveload_topicmodels import save_modelanddata, load_modelanddata
import datetime
import os
import time
import sys
import pandas as pd
import numpy as np
from collections import defaultdict, Counter

from pandarallel import pandarallel
pandarallel.initialize()


INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
print("getting data")
# get random samples from each party of similar sizes
datafiles = glob.glob("/data/shruti/ONR/big_data/Twitter/*.csv")
plist = ['_BJP.csv', '_INC.csv', '_AAP.csv']
pcount = defaultdict(list)
for d in datafiles:
    for p in plist:
        if p in d:
            pcount[p].append(d)
                
final_files = pcount
#final_files = defaultdict(list)
# for p in pcount:
#     try:
#         final_files[p] = sample(pcount[p], 1000)
#     except:
#         final_files[p] = sample(pcount[p], 430)
    
    
#read randomly sampled data into a dataframe
rowlist = []
for p in final_files.keys():
    party = p.replace("_", "").replace(".csv", "")
    for d in final_files[p]:
        with open(d, "r") as jsonfile:
            for line in jsonfile:
                job= json.loads(line)
                if "text" in job:
                    text = job['text']
                else:
                    text = None
                if "screen_name" in job:
                    sn = job['screen_name']
                else:
                    sn = None
                    
                if "lang" in job:
                    language = job['lang']
                else:
                    language = None
                    
                row = [sn, text, party, language]
                rowlist.append(row)

                
frame = pd.DataFrame(rowlist, columns=['screen_name','text','party', 'language'])
frame = frame.dropna(subset=['text'])

getting data


In [3]:
frame.head()

,screen_name,text,party,language
0,None,RT @BJP4MP: भाजपा प्रदेश अध्यक्ष श्री @vdsharm...,BJP,hi
1,None,जय श्री महाँकाल 🙏🚩 https://t.co/1EbAMWv902,BJP,ne
2,None,"RT @vaibhavpawarmp: बिल्कुल सही, इसीलिए बाबर द...",BJP,hi
3,None,"RT @narendramodi: Добро пожаловать, г-н презид...",BJP,cs
4,None,RT @vaibhavpawarmp: भारतीय जनता युवा मोर्चा मध...,BJP,hi


In [4]:
len(frame)

15100730

In [5]:
frame.to_csv("/data/shruti/ONR/small_data/Twitter_text/all_text.csv")

In [7]:
frame = frame.dropna(subset=['text'])

In [8]:
len(frame)

15100730

In [12]:
def remove_retweets(text):
    if "RT " in text:
    #if text.startswith("RT "):
        return 1

In [15]:
frame['is_retweet'] = frame['text'].parallel_apply(lambda x: remove_retweets(x))

In [16]:
frame.head()

,screen_name,text,party,language,is_retweet
0,None,RT @BJP4MP: भाजपा प्रदेश अध्यक्ष श्री @vdsharm...,BJP,hi,1.0
1,None,जय श्री महाँकाल 🙏🚩 https://t.co/1EbAMWv902,BJP,ne,NaN
2,None,"RT @vaibhavpawarmp: बिल्कुल सही, इसीलिए बाबर द...",BJP,hi,1.0
3,None,"RT @narendramodi: Добро пожаловать, г-н презид...",BJP,cs,1.0
4,None,RT @vaibhavpawarmp: भारतीय जनता युवा मोर्चा मध...,BJP,hi,1.0


In [17]:
data = frame.dropna(subset=['is_retweet'])
len(data)

8913413

In [18]:
len(data.drop_duplicates(subset=['text']))

3822026

In [19]:
8913413 - 3822026

5091387

In [25]:
unique_texts = list(set(data['text'].tolist()))

In [26]:
len(unique_texts)

3822026

In [27]:
def assign_unique_group(text):
    uid = unique_texts.index(text)
    return uid

In [ ]:
data['text_id'] = data['text'].parallel_apply(lambda x: assign_unique_group(x))

In [ ]:
data.to_csv("/data/shruti/ONR/small_data/Twitter_text/non_retweet_duplicate_text.csv")